- This notebook to get latent space of binary mixture dataset by using chemprop

In [7]:
import sys
sys.path.append("../src")
sys.path.append("../")

In [1]:
import os
import chemprop
from chemprop import data, featurizers, models, nn

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [3]:
import torch
import sys

In [8]:
from utils import normalize_concentrations

In [10]:
from chemprop_latent import chemprop_latent_space, build_mixture_latent_features_chemprop

In [9]:
df_mixture = pd.read_csv('df_mixture_20250502.csv')
# Normalize concentrations
conc_cols = [f'conc{i}' for i in range(1, 3)]
df_mixture_norm = normalize_concentrations(df_mixture, conc_cols)
df_mixture_norm.head()

,smi1,conc1,smi2,conc2,miscibility
0,C[C@H](CCC(=O)NCCC[N+](C)(C)CCCS(=O)(=O)[O-])[...,0.407074,CC(C)(C)CC(C)(C)C1=CC=C(C=C1)OCCO,0.592926,1
1,C[C@H](CCC(=O)NCCC[N+](C)(C)CCCS(=O)(=O)[O-])[...,0.784804,[C@H]1([C@@H]([C@@H](OC([C@H]1O)C(=O)O)O[C@H]2...,0.215196,1
2,C[C@H](CCC(=O)NCCC[N+](C)(C)CCCS(=O)(=O)[O-])[...,0.729393,C([C@@H]1[C@H]([C@@H]([C@H]([C@H](O1)O[C@@H]2[...,0.270607,1
3,C[C@H](CCC(=O)NCCC[N+](C)(C)CCCS(=O)(=O)[O-])[...,0.953164,C1[C@@H]2[C@@H]([C@H](O1)[C@H]([C@H](O2)O)OS(=...,0.046836,1
4,C[C@H](CCC(=O)NCCC[N+](C)(C)CCCS(=O)(=O)[O-])[...,0.962481,C1[C@@H]2[C@@H]([C@H](O1)[C@H]([C@H](O2)O)O)O[...,0.037519,1


In [ ]:
checkpoint_path='../src/models/chemprop/example_model_v2_regression_mol.ckpt'

In [12]:
x_latent, y = build_mixture_latent_features_chemprop(
    df=df_mixture_norm,
    smi_cols=["smi1", "smi2"],
    conc_cols=["conc1", "conc2"],
    target_col="miscibility",
    latent_fn=chemprop_latent_space,
    latent_fn_args={"checkpoint_path": checkpoint_path}
)

In [13]:
x_latent.shape

(1378, 300)

In [15]:
df_total = pd.concat([pd.DataFrame(x_latent),pd.DataFrame(y, columns=['miscibility'])], axis=1)
df_total.head()

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,miscibility
0,-0.036238,-0.023406,-0.002343,0.069807,-0.010298,0.031970,0.040516,-0.033724,-0.105037,-0.038822,...,0.089301,0.087046,-0.025358,-0.103743,0.135254,0.063807,-0.115770,0.038867,0.011449,1
1,-0.044862,-0.014836,-0.020091,0.060216,-0.027723,0.049013,0.042344,-0.005496,-0.109785,-0.028684,...,0.076220,0.090950,-0.027248,-0.093128,0.126671,0.071498,-0.118525,0.045216,0.007725,1
2,-0.051670,-0.008466,-0.030265,0.061587,-0.023505,0.049966,0.044878,-0.008011,-0.111385,-0.021101,...,0.071144,0.094368,-0.030387,-0.095236,0.131327,0.076321,-0.132885,0.044025,0.006640,1
3,-0.045585,-0.013987,-0.023957,0.062866,-0.031159,0.050780,0.044137,-0.010586,-0.109731,-0.025013,...,0.079209,0.087386,-0.027405,-0.095034,0.134498,0.071258,-0.129377,0.047157,0.009190,1
4,-0.045527,-0.013486,-0.023851,0.062846,-0.031196,0.050617,0.043903,-0.010788,-0.109916,-0.024579,...,0.079437,0.087866,-0.027837,-0.094904,0.134849,0.071392,-0.129961,0.046613,0.009391,1


In [16]:
df_total.to_csv('df_total_binary_chemprop_20250707.csv', index=False)